# Encapsulation in Python

---

## Table of Contents
1. What is Encapsulation?
2. Public, Protected, and Private Members
3. Name Mangling
4. Getters and Setters
5. Property Decorator
6. Read-Only and Write-Only Properties
7. Data Validation with Properties
8. Encapsulation Best Practices
9. Key Points
10. Practice Exercises

---

## 1. What is Encapsulation?

**Encapsulation** bundles data (attributes) and methods that operate on the data within a single unit (class), and restricts direct access to some components.

**Benefits:**
- **Data Hiding**: Internal state is protected from external access
- **Controlled Access**: Access through methods allows validation
- **Flexibility**: Internal implementation can change without affecting external code
- **Maintainability**: Easier to locate bugs and make changes

In [ ]:
# Without encapsulation - data can be corrupted
class BankAccountBad:
    def __init__(self, balance):
        self.balance = balance  # Public - can be modified directly

account = BankAccountBad(1000)
print(f"Initial balance: {account.balance}")

# Anyone can set invalid values!
account.balance = -5000  # No validation!
print(f"After direct modification: {account.balance}")

In [ ]:
# With encapsulation - data is protected
class BankAccountGood:
    def __init__(self, balance):
        self._balance = balance  # Protected by convention
    
    def deposit(self, amount):
        if amount > 0:
            self._balance += amount
            return True
        return False
    
    def withdraw(self, amount):
        if 0 < amount <= self._balance:
            self._balance -= amount
            return True
        return False
    
    def get_balance(self):
        return self._balance

account = BankAccountGood(1000)
print(f"Balance: {account.get_balance()}")
print(f"Withdraw 500: {account.withdraw(500)}")
print(f"Withdraw 1000: {account.withdraw(1000)}")
print(f"Final balance: {account.get_balance()}")

---

## 2. Public, Protected, and Private Members

Python uses naming conventions (not strict enforcement):

| Convention | Meaning | Access |
|------------|---------|--------|
| `name` | Public | Accessible everywhere |
| `_name` | Protected | Should only be accessed within class and subclasses |
| `__name` | Private | Name mangling applied, harder to access |

In [ ]:
# Public members
class Person:
    def __init__(self, name, age):
        self.name = name  # Public
        self.age = age    # Public

p = Person("Alice", 30)
print(f"Name: {p.name}")  # Direct access
print(f"Age: {p.age}")    # Direct access

# Can be modified directly
p.name = "Bob"
print(f"New name: {p.name}")

In [ ]:
# Protected members (single underscore)
class Employee:
    def __init__(self, name, salary):
        self.name = name       # Public
        self._salary = salary  # Protected (convention)
        self._ssn = "123-45-6789"  # Protected
    
    def get_salary(self):
        return self._salary

class Manager(Employee):
    def give_raise(self, amount):
        self._salary += amount  # Subclass can access protected

emp = Employee("Alice", 50000)
print(f"Salary via method: {emp.get_salary()}")

# Still accessible (Python trusts developers)
print(f"Direct access (not recommended): {emp._salary}")

In [ ]:
# Private members (double underscore)
class SecretKeeper:
    def __init__(self):
        self.public = "I'm public"
        self._protected = "I'm protected"
        self.__private = "I'm private"  # Name mangling
    
    def get_private(self):
        return self.__private
    
    def reveal_all(self):
        return f"Public: {self.public}, Protected: {self._protected}, Private: {self.__private}"

s = SecretKeeper()
print(f"Public: {s.public}")
print(f"Protected: {s._protected}")

# Private - raises AttributeError
try:
    print(s.__private)
except AttributeError as e:
    print(f"Error: {e}")

# Access via method
print(f"Via method: {s.get_private()}")

---

## 3. Name Mangling

Python transforms `__name` to `_ClassName__name` to avoid conflicts in subclasses.

In [ ]:
# Name mangling explained
class MyClass:
    def __init__(self):
        self.__secret = "hidden value"

obj = MyClass()

# View all attributes
print("All attributes:")
print([attr for attr in dir(obj) if not attr.startswith('__') or attr.startswith('_MyClass')])

# The mangled name
print(f"\nMangled name access: {obj._MyClass__secret}")

In [ ]:
# Name mangling prevents accidental override in subclasses
class Parent:
    def __init__(self):
        self.__value = "parent"
    
    def get_value(self):
        return self.__value

class Child(Parent):
    def __init__(self):
        super().__init__()
        self.__value = "child"  # This creates _Child__value, not _Parent__value
    
    def get_child_value(self):
        return self.__value

c = Child()
print(f"Parent's value: {c.get_value()}")       # parent
print(f"Child's value: {c.get_child_value()}")  # child

# Both exist!
print(f"\n_Parent__value: {c._Parent__value}")
print(f"_Child__value: {c._Child__value}")

In [ ]:
# Without name mangling - accidental override
class Parent2:
    def __init__(self):
        self._value = "parent"
    
    def get_value(self):
        return self._value

class Child2(Parent2):
    def __init__(self):
        super().__init__()
        self._value = "child"  # Overwrites parent's _value!

c2 = Child2()
print(f"Parent's value: {c2.get_value()}")  # child - overwritten!

---

## 4. Getters and Setters

In [ ]:
# Traditional getters and setters
class Person:
    def __init__(self, name, age):
        self._name = name
        self._age = age
    
    # Getter
    def get_name(self):
        return self._name
    
    # Setter with validation
    def set_name(self, value):
        if not value or not isinstance(value, str):
            raise ValueError("Name must be a non-empty string")
        self._name = value
    
    def get_age(self):
        return self._age
    
    def set_age(self, value):
        if not isinstance(value, int) or value < 0:
            raise ValueError("Age must be a non-negative integer")
        self._age = value

p = Person("Alice", 30)
print(f"Name: {p.get_name()}")
print(f"Age: {p.get_age()}")

p.set_name("Bob")
p.set_age(25)
print(f"\nNew name: {p.get_name()}")
print(f"New age: {p.get_age()}")

try:
    p.set_age(-5)
except ValueError as e:
    print(f"\nError: {e}")

---

## 5. Property Decorator

The Pythonic way to implement getters and setters.

In [ ]:
# Using @property - Pythonic way
class Person:
    def __init__(self, name, age):
        self._name = name
        self._age = age
    
    @property
    def name(self):
        """Get the person's name."""
        return self._name
    
    @name.setter
    def name(self, value):
        if not value or not isinstance(value, str):
            raise ValueError("Name must be a non-empty string")
        self._name = value
    
    @property
    def age(self):
        return self._age
    
    @age.setter
    def age(self, value):
        if not isinstance(value, int) or value < 0:
            raise ValueError("Age must be a non-negative integer")
        self._age = value

p = Person("Alice", 30)

# Access like attributes, but uses methods behind the scenes
print(f"Name: {p.name}")
print(f"Age: {p.age}")

# Set like attributes, but validation runs
p.name = "Bob"
p.age = 25
print(f"\nNew name: {p.name}")
print(f"New age: {p.age}")

try:
    p.age = -5
except ValueError as e:
    print(f"\nError: {e}")

In [ ]:
# Property with property() function (alternative syntax)
class Circle:
    def __init__(self, radius):
        self._radius = radius
    
    def _get_radius(self):
        return self._radius
    
    def _set_radius(self, value):
        if value < 0:
            raise ValueError("Radius cannot be negative")
        self._radius = value
    
    def _del_radius(self):
        print("Deleting radius")
        del self._radius
    
    radius = property(_get_radius, _set_radius, _del_radius, "Radius of the circle")

c = Circle(5)
print(f"Radius: {c.radius}")
c.radius = 10
print(f"New radius: {c.radius}")
print(f"Docstring: {Circle.radius.__doc__}")

---

## 6. Read-Only and Write-Only Properties

In [ ]:
# Read-only property (no setter)
class Circle:
    def __init__(self, radius):
        self._radius = radius
    
    @property
    def radius(self):
        return self._radius
    
    @radius.setter
    def radius(self, value):
        if value < 0:
            raise ValueError("Radius cannot be negative")
        self._radius = value
    
    @property
    def area(self):  # Read-only - calculated
        import math
        return math.pi * self._radius ** 2
    
    @property
    def diameter(self):  # Read-only - calculated
        return self._radius * 2

c = Circle(5)
print(f"Radius: {c.radius}")
print(f"Area: {c.area:.2f}")
print(f"Diameter: {c.diameter}")

# Can modify radius
c.radius = 10
print(f"\nNew area: {c.area:.2f}")

# Cannot modify area directly
try:
    c.area = 100
except AttributeError as e:
    print(f"Error: {e}")

In [ ]:
# Write-only property (useful for passwords)
import hashlib

class User:
    def __init__(self, username):
        self.username = username
        self._password_hash = None
    
    @property
    def password(self):
        raise AttributeError("Password is write-only")
    
    @password.setter
    def password(self, value):
        # Store hash, not plain password
        self._password_hash = hashlib.sha256(value.encode()).hexdigest()
    
    def check_password(self, password):
        return self._password_hash == hashlib.sha256(password.encode()).hexdigest()

user = User("alice")
user.password = "secret123"  # Sets password hash

print(f"Password check 'secret123': {user.check_password('secret123')}")
print(f"Password check 'wrong': {user.check_password('wrong')}")

# Cannot read password
try:
    print(user.password)
except AttributeError as e:
    print(f"Error: {e}")

---

## 7. Data Validation with Properties

In [ ]:
# Comprehensive validation example
class Product:
    def __init__(self, name, price, quantity):
        self.name = name
        self.price = price
        self.quantity = quantity
    
    @property
    def name(self):
        return self._name
    
    @name.setter
    def name(self, value):
        if not isinstance(value, str) or len(value) < 2:
            raise ValueError("Name must be at least 2 characters")
        self._name = value.strip().title()
    
    @property
    def price(self):
        return self._price
    
    @price.setter
    def price(self, value):
        if not isinstance(value, (int, float)) or value < 0:
            raise ValueError("Price must be a non-negative number")
        self._price = round(float(value), 2)
    
    @property
    def quantity(self):
        return self._quantity
    
    @quantity.setter
    def quantity(self, value):
        if not isinstance(value, int) or value < 0:
            raise ValueError("Quantity must be a non-negative integer")
        self._quantity = value
    
    @property
    def total_value(self):
        return self._price * self._quantity

# Valid product
p = Product("  laptop  ", 999.999, 5)
print(f"Name: {p.name}")
print(f"Price: ${p.price}")
print(f"Quantity: {p.quantity}")
print(f"Total value: ${p.total_value}")

# Invalid attempts
try:
    p.price = -100
except ValueError as e:
    print(f"\nPrice error: {e}")

try:
    p.name = "A"
except ValueError as e:
    print(f"Name error: {e}")

In [ ]:
# Email validation example
import re

class Contact:
    def __init__(self, name, email, phone):
        self.name = name
        self.email = email
        self.phone = phone
    
    @property
    def email(self):
        return self._email
    
    @email.setter
    def email(self, value):
        pattern = r'^[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}$'
        if not re.match(pattern, value):
            raise ValueError(f"Invalid email format: {value}")
        self._email = value.lower()
    
    @property
    def phone(self):
        return self._phone
    
    @phone.setter
    def phone(self, value):
        # Remove non-digits
        digits = re.sub(r'\D', '', value)
        if len(digits) != 10:
            raise ValueError("Phone must have 10 digits")
        self._phone = f"({digits[:3]}) {digits[3:6]}-{digits[6:]}"

c = Contact("Alice", "ALICE@Example.COM", "123-456-7890")
print(f"Name: {c.name}")
print(f"Email: {c.email}")
print(f"Phone: {c.phone}")

try:
    c.email = "invalid-email"
except ValueError as e:
    print(f"\nError: {e}")

---

## 8. Encapsulation Best Practices

In [ ]:
# Best Practice 1: Start with public, make private if needed
class ConfigurationV1:
    def __init__(self):
        self.debug_mode = False  # Start public
        self.log_level = "INFO"

# Later, if you need validation:
class ConfigurationV2:
    VALID_LOG_LEVELS = ["DEBUG", "INFO", "WARNING", "ERROR"]
    
    def __init__(self):
        self._debug_mode = False
        self._log_level = "INFO"
    
    @property
    def log_level(self):
        return self._log_level
    
    @log_level.setter
    def log_level(self, value):
        if value not in self.VALID_LOG_LEVELS:
            raise ValueError(f"Invalid log level. Choose from {self.VALID_LOG_LEVELS}")
        self._log_level = value

# API stays the same!
config = ConfigurationV2()
config.log_level = "DEBUG"  # Works the same as V1
print(f"Log level: {config.log_level}")

In [ ]:
# Best Practice 2: Use properties for computed attributes
class Rectangle:
    def __init__(self, width, height):
        self._width = width
        self._height = height
    
    @property
    def width(self):
        return self._width
    
    @width.setter
    def width(self, value):
        if value <= 0:
            raise ValueError("Width must be positive")
        self._width = value
    
    @property
    def height(self):
        return self._height
    
    @height.setter
    def height(self, value):
        if value <= 0:
            raise ValueError("Height must be positive")
        self._height = value
    
    # Computed properties - always up to date
    @property
    def area(self):
        return self._width * self._height
    
    @property
    def perimeter(self):
        return 2 * (self._width + self._height)
    
    @property
    def is_square(self):
        return self._width == self._height

r = Rectangle(4, 5)
print(f"Area: {r.area}, Perimeter: {r.perimeter}, Square: {r.is_square}")
r.width = 5
print(f"Area: {r.area}, Perimeter: {r.perimeter}, Square: {r.is_square}")

In [ ]:
# Best Practice 3: Encapsulate collections
class Playlist:
    def __init__(self, name):
        self.name = name
        self._songs = []  # Protected list
    
    @property
    def songs(self):
        # Return a copy to prevent external modification
        return self._songs.copy()
    
    @property
    def count(self):
        return len(self._songs)
    
    def add_song(self, song):
        if song not in self._songs:
            self._songs.append(song)
    
    def remove_song(self, song):
        if song in self._songs:
            self._songs.remove(song)

playlist = Playlist("My Favorites")
playlist.add_song("Song A")
playlist.add_song("Song B")

# Get songs
songs = playlist.songs
print(f"Songs: {songs}")

# Modifying the returned list doesn't affect playlist
songs.append("Song C")
print(f"After external modification: {playlist.songs}")

---

## 9. Key Points

1. **Encapsulation**: Bundle data and methods, restrict access
2. **Public (name)**: Accessible everywhere
3. **Protected (_name)**: Convention - internal use only
4. **Private (__name)**: Name mangling - `_ClassName__name`
5. **Properties**: Pythonic way to implement getters/setters
6. **Read-only**: Property without setter (computed attributes)
7. **Write-only**: Property with setter that raises on get (passwords)
8. **Validation**: Use setters to validate data before storing
9. **Collections**: Return copies to prevent external modification
10. **Start Public**: Add encapsulation when needed, API stays same

---

## 10. Practice Exercises

In [ ]:
# Exercise 1: Create a Temperature class with:
# - Private _celsius attribute
# - celsius property (validates >= -273.15)
# - fahrenheit property (read/write, converts)
# - kelvin property (read-only)

class Temperature:
    pass

# Test:
# t = Temperature(25)
# print(t.celsius)     # 25
# print(t.fahrenheit)  # 77.0
# t.fahrenheit = 100
# print(t.celsius)     # ~37.78

In [ ]:
# Exercise 2: Create a BankAccount class with:
# - Private __balance and __pin
# - balance property (read-only)
# - deposit(amount) and withdraw(amount, pin) methods
# - Pin verification for withdrawals

class BankAccount:
    pass

# Test:
# acc = BankAccount("1234", 1000)
# print(acc.balance)  # 1000
# acc.withdraw(100, "1234")  # Works
# acc.withdraw(100, "0000")  # Fails

In [ ]:
# Exercise 3: Create an Employee class with:
# - name (validated: non-empty string)
# - email (validated: proper format)
# - salary (validated: positive, manager approval for > 100000)
# - employee_id (read-only, auto-generated)

class Employee:
    pass

# Test:
# e = Employee("Alice", "alice@company.com", 50000)
# print(e.employee_id)  # Auto-generated
# e.salary = 60000  # Works
# e.salary = 150000  # Needs approval

In [ ]:
# Exercise 4: Create a ShoppingCart class with:
# - Protected _items dict {product: quantity}
# - items property returns copy
# - add_item, remove_item, update_quantity methods
# - total_items and unique_items properties

class ShoppingCart:
    pass

# Test:
# cart = ShoppingCart()
# cart.add_item("Apple", 3)
# cart.add_item("Banana", 2)
# print(cart.total_items)   # 5
# print(cart.unique_items)  # 2

In [ ]:
# Exercise 5: Create a SecureConfig class with:
# - Public settings dict for non-sensitive data
# - Private __secrets dict for sensitive data (API keys, passwords)
# - get_setting, set_setting methods for public
# - set_secret (write-only), check_secret methods for private

class SecureConfig:
    pass

# Test:
# config = SecureConfig()
# config.set_setting("theme", "dark")
# config.set_secret("api_key", "secret123")
# print(config.get_setting("theme"))  # dark
# config.check_secret("api_key", "secret123")  # True

---

## Solutions

In [ ]:
# Solution 1:
class Temperature:
    def __init__(self, celsius):
        self.celsius = celsius
    
    @property
    def celsius(self):
        return self._celsius
    
    @celsius.setter
    def celsius(self, value):
        if value < -273.15:
            raise ValueError("Temperature below absolute zero")
        self._celsius = value
    
    @property
    def fahrenheit(self):
        return self._celsius * 9/5 + 32
    
    @fahrenheit.setter
    def fahrenheit(self, value):
        self.celsius = (value - 32) * 5/9
    
    @property
    def kelvin(self):
        return self._celsius + 273.15

t = Temperature(25)
print(f"Celsius: {t.celsius}")
print(f"Fahrenheit: {t.fahrenheit}")
print(f"Kelvin: {t.kelvin}")
t.fahrenheit = 100
print(f"\nAfter setting 100F, Celsius: {t.celsius:.2f}")

In [ ]:
# Solution 2:
class BankAccount:
    def __init__(self, pin, initial_balance=0):
        self.__pin = pin
        self.__balance = initial_balance
    
    @property
    def balance(self):
        return self.__balance
    
    def deposit(self, amount):
        if amount > 0:
            self.__balance += amount
            return f"Deposited ${amount}. Balance: ${self.__balance}"
        raise ValueError("Deposit amount must be positive")
    
    def withdraw(self, amount, pin):
        if pin != self.__pin:
            raise ValueError("Invalid PIN")
        if amount > self.__balance:
            raise ValueError("Insufficient funds")
        if amount <= 0:
            raise ValueError("Amount must be positive")
        self.__balance -= amount
        return f"Withdrew ${amount}. Balance: ${self.__balance}"

acc = BankAccount("1234", 1000)
print(f"Balance: ${acc.balance}")
print(acc.deposit(500))
print(acc.withdraw(200, "1234"))

try:
    acc.withdraw(100, "0000")
except ValueError as e:
    print(f"Error: {e}")

In [ ]:
# Solution 3:
import re

class Employee:
    _id_counter = 0
    SALARY_THRESHOLD = 100000
    
    def __init__(self, name, email, salary):
        Employee._id_counter += 1
        self._employee_id = f"EMP{Employee._id_counter:04d}"
        self.name = name
        self.email = email
        self._salary = salary
    
    @property
    def employee_id(self):
        return self._employee_id
    
    @property
    def name(self):
        return self._name
    
    @name.setter
    def name(self, value):
        if not value or not isinstance(value, str):
            raise ValueError("Name must be non-empty string")
        self._name = value.strip()
    
    @property
    def email(self):
        return self._email
    
    @email.setter
    def email(self, value):
        pattern = r'^[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}$'
        if not re.match(pattern, value):
            raise ValueError("Invalid email format")
        self._email = value.lower()
    
    @property
    def salary(self):
        return self._salary
    
    @salary.setter
    def salary(self, value):
        if value <= 0:
            raise ValueError("Salary must be positive")
        if value > self.SALARY_THRESHOLD:
            print(f"Warning: Salary ${value} requires manager approval")
        self._salary = value

e = Employee("Alice", "alice@company.com", 50000)
print(f"ID: {e.employee_id}, Name: {e.name}, Salary: ${e.salary}")
e.salary = 150000

In [ ]:
# Solution 4:
class ShoppingCart:
    def __init__(self):
        self._items = {}
    
    @property
    def items(self):
        return self._items.copy()
    
    @property
    def total_items(self):
        return sum(self._items.values())
    
    @property
    def unique_items(self):
        return len(self._items)
    
    def add_item(self, product, quantity=1):
        if quantity <= 0:
            raise ValueError("Quantity must be positive")
        self._items[product] = self._items.get(product, 0) + quantity
    
    def remove_item(self, product):
        if product in self._items:
            del self._items[product]
    
    def update_quantity(self, product, quantity):
        if product not in self._items:
            raise ValueError(f"Product '{product}' not in cart")
        if quantity <= 0:
            self.remove_item(product)
        else:
            self._items[product] = quantity

cart = ShoppingCart()
cart.add_item("Apple", 3)
cart.add_item("Banana", 2)
cart.add_item("Apple", 2)
print(f"Items: {cart.items}")
print(f"Total items: {cart.total_items}")
print(f"Unique items: {cart.unique_items}")

In [ ]:
# Solution 5:
import hashlib

class SecureConfig:
    def __init__(self):
        self.settings = {}
        self.__secrets = {}
    
    def get_setting(self, key):
        return self.settings.get(key)
    
    def set_setting(self, key, value):
        self.settings[key] = value
    
    def set_secret(self, key, value):
        # Store hash of secret
        self.__secrets[key] = hashlib.sha256(value.encode()).hexdigest()
    
    def check_secret(self, key, value):
        if key not in self.__secrets:
            return False
        return self.__secrets[key] == hashlib.sha256(value.encode()).hexdigest()
    
    def has_secret(self, key):
        return key in self.__secrets

config = SecureConfig()
config.set_setting("theme", "dark")
config.set_setting("language", "en")
config.set_secret("api_key", "secret123")
config.set_secret("db_password", "password456")

print(f"Theme: {config.get_setting('theme')}")
print(f"Has api_key: {config.has_secret('api_key')}")
print(f"Check api_key 'secret123': {config.check_secret('api_key', 'secret123')}")
print(f"Check api_key 'wrong': {config.check_secret('api_key', 'wrong')}")